In [1]:
%run ../course_helpers/init_working_environment.ipynb

Loaded 1565 rows from /workspaces/mlops-course/data/comments.csv
First ten rows of the dataframe `df`:


,content,is_spam
0,Best Music Ever!!!,0
1,please look up DHG SONGS this is my playlist with a bunch of amazing songs,1
2,just :( superr!!!,0
3,Check out this playlist on YouTube:,1
4,subscribed :) btw you have a good style keep it up brother :)),1
5,she is horrible at acting. cringe-worhty.,0
6,https://m.facebook.com/story.php?story_fbid=764484966942313&amp;id=754989901225153&amp;ref=stream gf,1
7,WOw,0
8,I loved this song when I was in my teenage years!,0
9,Where did she find all that make up in a freakin jungle?!,0


In [2]:
my_experiment = mlflow_connect()

OK - mlflow server is up and running. Setting Tracking URI to http://localhost:5000. Setting Experiment to 'Spam Detection'


In [12]:
import mlflow

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Initialize Random Forest model with best parameters
count_vectorizer = CountVectorizer()
best_params = {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
rf = RandomForestClassifier(random_state=42, **best_params)

X = df[["content"]]
y = df["is_spam"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

col_trans = ColumnTransformer(
        transformers=[
            ('text_data', count_vectorizer, 'content')  # Apply CountVectorizer directly to the 'text' column
        ],
        remainder='drop'
    )

mlflow.sklearn.autolog(disable=True)
pipe = Pipeline([('Pandas Count Vectorizer', col_trans), ('RandomForest', rf)])
pipe.fit(X_train, y_train)

MODEL_PATH=f"{PROJECT_ROOT_DIR}/models/spam_pipeline_model_pandas"
import shutil
shutil.rmtree(MODEL_PATH, ignore_errors=True)
model_info = mlflow.sklearn.save_model(pipe, MODEL_PATH)
print(f'Model saved to {MODEL_PATH}')

Model saved to /workspaces/mlops-course/models/spam_pipeline_model_pandas


In [13]:
messages = ["Click this link and subscribe to my channel!", "This is the best ever video about MLflow!"]

artifact_path = f"runs:/{run.info.run_id}/spam_pipeline_model"
print(f"Loading model from {artifact_path}")
print()

reloaded_model = mlflow.sklearn.load_model(artifact_path)

predictions = reloaded_model.predict(X_test)

for message, prediction in zip(messages, predictions):
    print(f"Comment: {message} \t| Spam? (prediction): {prediction}")


NameError: name 'run' is not defined